# Clase Práctica #10 (Compilación)

En esta clase estaremos implementando un **generador de parsers SLR(1)**. Nos apoyaremos en la API de lenguajes que llevamos implementando desde el semestre anterior.

Comencemos por importar la clase `Grammar`.

In [ ]:
from cmp.pycompiler import Grammar

Como de costumbre trabajaremos sobre un lenguaje de expresiones aritméticas básicas. Sin embargo, esta vez podemos finalmente usar la gramática natural de expresiones puesto que, como estudiamos en conferencia, los parser SLR(1) a diferencia de los LL(1) no son susceptibles a la presencia de recursión izquierda ni de prefijos comunes. Esto posibilita que la asociatividad a la izquierda de muchos de los operadores pueda ser representado sin problema.

In [ ]:
G = Grammar()
E = G.NonTerminal('E', True)
T,F = G.NonTerminals('T F')
plus, minus, star, div, opar, cpar, num = G.Terminals('+ - * / ( ) int')

E %= E + plus + T | T # | E + minus + T 
T %= T + star + F | F # | T + div + F
F %= num | opar + E + cpar

print(G)

## Items

Se provee una implementación de la clase `Item` para modelar los Items LR(0).

In [ ]:
from cmp.pycompiler import Item

Los items se construyen a partir de una producción.

In [ ]:
prod = E.productions[0]
print('Posibles items de',repr(prod))

for x in range(len(prod.Right)+1):
    item = Item(prod, x)
    print('---------------------------------')
    print('item:', item)
    print('item.pos:', item.pos)
    print('item.IsReduceItem:', item.IsReduceItem)
    print('item.NextSymbol:', item.NextSymbol)
    print('item.NextItem():', item.NextItem())

## Autómata LR(0)

Usaremos la implementación por referencia de autómata. Recordemos que bajo esta representación, los estados están conectados por referencias y el autómata resulta de seleccionar uno de ellos como raíz y expandir el grafo a partir de él.

In [ ]:
from cmp.automata import State, lr0_formatter

La API de gramáticas provee la función `AugmentedGrammar` que construye una nueva gramática a partir de aumentar otra. Los símbolos y producciones de la gramática original se mantienen con las mismas referencias. Si el símbolo distinguido de la gramática a aumentar nunca aparece en parte derecha se devuelve la propia gramática. Es posible forzar el aumento de la gramática al incluir el argumento `force=True` al llamar a la función.

In [ ]:
GG = G.AugmentedGrammar()

assert len(GG.startSymbol.productions) == 1
start_production = GG.startSymbol.productions[0]
start_production

In [ ]:
start_item = Item(start_production, 0)
start_item

### Construcción del autómata LR(0)

Implementemos el algoritmo para construir la versión no determinista del autómata LR(0). Recordemos de conferencia que:
- Cada item representa un estado.
- El estado inicial es representado por el item $S' \to .S$
- Todos los estados son finales: _Todo prefijo de un prefijo viable es un prefijo viable_.
    - Una cadena no es un prefijo viable si el autómata se traba.
- Función de transición:
    - $(X \to \alpha . c \beta) \longrightarrow^{c} (X \to \alpha c . \beta)$, con $c \in V_T$
    - $(X \to \alpha . Y \beta) \longrightarrow^{Y} (X \to \alpha Y . \beta)$, con $Y \in V_N$
    - $(X \to \alpha . Y \beta) \longrightarrow^{\epsilon} (Y \to .\delta)$, con $Y \in V_N$

In [ ]:
# NOTA: use `symbol.Name` al hacer las transiciones, no directamente `symbol`.

def build_LR0_automaton(G):
    assert len(G.startSymbol.productions) == 1, 'Grammar must be augmented'

    start_production = G.startSymbol.productions[0]
    start_item = Item(start_production, 0)

    automaton = State(start_item, True)

    pending = [ start_item ]
    visited = { start_item: automaton }

    while pending:
        current_item = pending.pop()
        if current_item.IsReduceItem:
            continue
        
        # Your code here!!! (Decide which transitions to add)

        current_state = visited[current_item]
        # Your code here!!! (Add the decided transitions)
    return automaton

Al correr el algoritmo debemos obtener la versión no determinista del autómata. Recordemos que este autómata reconoce el lenguaje de los prefijos viables de una gramática: cadenas que pueden ocurrir en la pila durante el parseo de una cadena válida.

In [ ]:
automaton = build_LR0_automaton(GG)

assert automaton.recognize('E')
assert automaton.recognize('T*F')
assert automaton.recognize(['E', '+', 'int'])
assert not automaton.recognize('E*F')

automaton.set_formatter(lr0_formatter)

Para construir la versión determinista del autómata LR(0) simplemente aplicaremos el método `to_deterministic` que implementamos en clases anteriores. En conferencia estudiamos que es posible construir el autómata determinista directamente a partir de calcular la **Colección Canónica de Items LR(0)**. Esta variante queda propuesta a implementar como estudio individual.

In [ ]:
automaton.to_deterministic(lr0_formatter)

## Parsers Shift-Reduce

Un parser *shift-reduce* es un mecanismo de parsing que cuenta con las siguientes estructuras:

- Una pila de símbolos `S`.
- Una secuencia de terminales `T`.

> Denotamos el estado del parser como $\alpha|\omega$, con $S = \alpha$ y $T = \omega$.

Y las operaciones siguientes:

- **shift**: Si el parser se encuentra en un estado $\alpha | c \omega$, entonces tras aplicar una operación _shift_ pasa al estado $\alpha c | \omega$.
- **reduce**: Si el parser se encuentra en un estado $\alpha \beta | \omega$, y $X \rightarrow \beta$ es una producción, entonces tras aplicar una operación _reduce_ $T \rightarrow \beta$ pasa al estado $\alpha X | \omega$.

Podemos definir entonces el proceso de parsing como:

> Sea $S = \emptyset$ la pila inicial, $T = \omega \$$ la cadena a reconocer, y $E$ el símbolo inicial, un parser shift-reduce reconoce esta cadena si y solo si existe una secuencia de operaciones **shift** y **reduce** tal que tras aplicarlas se obtiene $S = E$ y $T = \$$.

Todos los algoritmos de parsing que estudiaremos en este semestre están basados en esta arquitectura. La diferencia entre ellos radica justamente en la forma en que deciden entre hacer _shift_ o _reduce_.

Para implementarlos, nos apoyaremos en una representación uniforme: tabla **Acción-Goto**, la cual sigue la siguiente estructura:


          ________ _______________________ ___________
         |________|_________ACTION________|___GOTO____|
         | Estado | +   *   (   )  int  $ | E   T   F |
         |--------|--- --- --- --- --- ---|--- --- ---|
         |   ...  |          ...          |    ...    |
         |________|_______________________|___________|

donde para todo $I_i$ estado de la Colección Canónica, $c \in V_T \cup \{ \$ \}$ y $X,Y \in V_N$.

- $ACTION[I_i, c] \in \{ `S_k`, `R_k`, `OK` \}$
- $GOTO[I_i, Y] \in \{ 1...N \}$


In [ ]:
class ShiftReduceParser:
    SHIFT = 'SHIFT'
    REDUCE = 'REDUCE'
    OK = 'OK'
    
    def __init__(self, G, verbose=False):
        self.G = G
        self.verbose = verbose
        self.action = {}
        self.goto = {}
        self._build_parsing_table()
    
    def _build_parsing_table(self):
        raise NotImplementedError()

    def __call__(self, w):
        stack = [ 0 ]
        cursor = 0
        output = []
        
        while True:
            state = stack[-1]
            lookahead = w[cursor]
            if self.verbose: print(stack, '<---||--->', w[cursor:])
                
            # Your code here!!! (Detect error)
            
            action, tag = self.action[state, lookahead]
            # Your code here!!! (Shift case)
            # Your code here!!! (Reduce case)
            # Your code here!!! (OK case)
            # Your code here!!! (Invalid case)


### Cómo llena la tabla un parser SLR(1)?

- **Sea** $X \to \alpha .c \omega$ un item del estado $I_i$ y $Goto(I_i,c) = I_j$. **Entonces** $ACTION[I_i,c] = `S_j`$.

- **Sea** $X \to \alpha .$ un item del estado $I_i$ y $c \in FOLLOW(X)$. **Entonces** $ACTION[I_i,c] = `R_k`$ (producción `k` es $X \to \alpha$).

- **Sea** $I_i$ el estado que contiene el item $S' \to S.$ ($S'$ distinguido). **Entonces** $ACTION[I_i,\$] = `OK`$.

- **Sea** $X \to \alpha .Y \omega$ item del estado $I_i$ y $Goto(I_i,Y) = I_j$. **Entonces** $GOTO[I_i,Y] = j$.

In [ ]:
from cmp.tools.parsing import compute_firsts, compute_follows

class SLR1Parser(ShiftReduceParser):

    def _build_parsing_table(self):
        G = self.G.AugmentedGrammar(True)
        firsts = compute_firsts(G)
        follows = compute_follows(G, firsts)
        
        automaton = build_LR0_automaton(G).to_deterministic()
        for i, node in enumerate(automaton):
            if self.verbose: print(i, '\t', '\n\t '.join(str(x) for x in node.state), '\n')
            node.idx = i

        for node in automaton:
            idx = node.idx
            for state in node.state:
                item = state.state
                # Your code here!!!
                # - Fill `self.Action` and `self.Goto` according to `item`)
                # - Feel free to use `self._register(...)`)
    
    @staticmethod
    def _register(table, key, value):
        assert key not in table or table[key] == value, 'Shift-Reduce or Reduce-Reduce conflict!!!'
        table[key] = value

## Probando

Construyamos un parser SLR(1) para la gramática de las expresiones aritméticas.

In [ ]:
parser = SLR1Parser(G, verbose=True)

### Tablas

Para visualizar las tablas Action y Goto usaremos la clase `DataFrame` de `pandas`.

In [ ]:
from pandas import DataFrame

def encode_value(value):
    try:
        action, tag = value
        if action == ShiftReduceParser.SHIFT:
            return 'S' + str(tag)
        elif action == ShiftReduceParser.REDUCE:
            return repr(tag)
        elif action ==  ShiftReduceParser.OK:
            return action
        else:
            return value
    except TypeError:
        return value

def table_to_dataframe(table):
    d = {}
    for (state, symbol), value in table.items():
        value = encode_value(value)
        try:
            d[state][symbol] = value
        except KeyError:
            d[state] = { symbol: value }

    return DataFrame.from_dict(d, orient='index', dtype=str)

Recordemos que:

- Debe haber a lo sumo una opción en cada celda.

- Deben aparecer todos los estados (salvo $I_0$) entre **ACTION** y **GOTO**.

- Deben aparecer todas las producciones entre los $R_k$ de **ACTION**.

In [ ]:
display(table_to_dataframe(parser.action))
display(table_to_dataframe(parser.goto))

### Parseando ...

Trabajemos sobre la cadena `int + int * int`. Si el parser está correctamente implementado deberíamos obtener una derivación extrema derecha en reverso que parta de la oración y llegue al símbolo distinguido.

In [ ]:
derivation = parser([num, plus, num, star, num, G.EOF])

assert str(derivation) == '[F -> int, T -> F, E -> T, F -> int, T -> F, F -> int, T -> T * F, E -> E + T]'

derivation

## Propuestas

- Complete el pipeline de evaluación. Observe que esta vez la gramática asocia naturalmente a la izquierda, lo cual debe simplificar la implementación de las reglas semánticas.

- Construya directamente la versión determinista del autómata LR(0).

- Explore otras gramáticas.

In [ ]:
TESTING = False
if TESTING:
    GG = Grammar()

    S = GG.NonTerminal('S', True)
    V,A,E = GG.NonTerminals('V A E')
    i, equal, obrac, cbrac, plus = GG.Terminals('i = [ ] +')

    S %= V + equal + E
    V %= i | A + obrac + E + cbrac
    A %= i
    E %= E + plus + V | V

    GG = GG.AugmentedGrammar(True)
    automaton = build_LR0_automaton(GG)
    display(automaton.set_formatter(lr0_formatter))
    display(automaton.to_deterministic(lr0_formatter))